# Demoing GPT to BlogPost

In [1]:
!pip install --upgrade openai

   ---------------------------------------- 0.0/337.0 kB ? eta -:--:--
   --------------------------------------- 337.0/337.0 kB 10.2 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.37.0
    Uninstalling openai-1.37.0:
      Successfully uninstalled openai-1.37.0


In [4]:
from openai import OpenAI
import random
import os
from dotenv import load_dotenv
import json

load_dotenv()
gpt = os.getenv('gpt_token')
org = os.getenv('gpt_org')
client = OpenAI(api_key=gpt, organization=org)

def read_jsonl(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return [json.loads(line) for line in file]

examples = read_jsonl("../example_docs/blog_posts.jsonl")
sample = []
for i in range(0,9):
    index = random.randint(0, len(examples))
    sample.append(examples[index])
    
examples_str = '\n'.join(json.dumps(example, indent=4) for example in sample[0:8])

master_prompt = """
            You are a real-estate blog writer and want to provide readers about real estate topics that are occurring now in the year 2024 
            and also retaining information from the past information on real estate. Given examples of blog posts and also extra data accumulated from external
            sources, you will write a unique real estate blog post, making use of SEO keywords to make the blog post stand out. In the blog post itself,
            you will provide:
                - Title: A title of what will covered in the blog post
                - Content: The content of the blog post. Write the blog post in markdown. You do not have to tell me it is in markdown, just please
                           write the blog post in markdown. It should consist of at least 5000 words. Make sure that there is a lot to say 
                           within categories and subcategories, making it an information read for the reader
                - SEO Terms: A list of the SEO keywords used in the blog post

            When writing a real estate blog post, make sure that it covers at least one of these themes and topics 
            (pick and choose a random topic from the list provided below):
                1. Market Trends and Analysis
                   - Current real estate market conditions
                   - Housing market predictions
                   - Economic factors affecting real estate
                   - Local market updates
                
                2. Home Buying and Selling Tips
                   - Step-by-step guides for first-time homebuyers
                   - Tips for selling a home quickly and for the best price
                   - Advice on choosing a real estate agent
                   - Understanding mortgage options
                
                3. Home Improvement and Maintenance
                   - DIY home improvement projects
                   - Tips for increasing home value
                   - Seasonal maintenance checklists
                   - Energy-efficient home upgrades
                
                4. Interior Design and Staging
                   - Home staging tips for sellers
                   - Interior design trends
                   - Space-saving ideas for small homes
                   - Color schemes and decorating tips
                
                5. Neighborhood and Community Spotlights
                   - Profiles of different neighborhoods
                   - Community amenities and local attractions
                   - School district information
                   - Local business reviews
                
                6. Investment and Financial Advice
                   - Real estate investment strategies
                   - Tips for becoming a landlord
                   - Understanding property taxes
                   - Financial planning for buying a home
                
                7. Legal and Regulatory Issues
                   - Understanding real estate contracts
                   - Changes in real estate laws and regulations
                   - Tenant rights and landlord responsibilities
                   - Zoning laws and property usage
                
                8. Sustainability and Green Living
                   - Eco-friendly home features
                   - Sustainable building practices
                   - Benefits of green homes
                   - Solar energy and other renewable resources
                
                9. Lifestyle and Homeownership
                   - Balancing homeownership with lifestyle choices
                   - Home office setups and work-from-home tips
                   - Pet-friendly home ideas
                   - Family-friendly home features
                
                10. Technology in Real Estate
                    - Smart home technology
                    - Virtual tours and augmented reality in home buying
                    - Online real estate platforms
                    - Home security systems
                
                11. Case Studies and Success Stories
                    - Personal stories of buying and selling homes
                    - Profiles of successful real estate investors
                    - Unique home renovations and builds
                    - Celebrity real estate stories
            Depending on what is asked of you, based on how many blogposts the client requests from, you will always default to one blog post.
            If the user asks for a specific amount, then provide that amount of blogposts in a listed array. However, in the output, you should respond
            with this structure:
            {
               "title": {Whatever the title of the blog post is},
               "content": {Whatever the content of the blog post is},
               "seo_terms": [],
            }
            Please make sure that you follow the structure above and that something like seo terms is not found within "content", but should rather be 
            in the "seo_terms" key of the object body. I REPEAT DO NOT WRITE THE SEO TERMS WITHIN THE CONTENT IN MARKDOWN. IT SHOULD ONLY BE SEEN
            WITHIN "seo_terms". Please make sure that you are not using external links and link them inside the blog post you generate. 
            Please do not mention anything promotional or any footers regarding where the blog post is getting its data from, 
            how it's written, or anything not related to what a blog post is supposed to write. Please do not write a table of contents. 
            You are simply writing content. Please also make use of real estate SEO terms from example blog posts that may be given from the user.
            """
user_prompt = f"""
            Hello, I would like you to help me generate a unique real estate blog post. Here are examples of how real estate blog posts that I saw
            and would hope you would be able to come up with something unique that consists of similar topics of the examples. Here are the examples:
            {examples_str}. Follow the structure of the examples, and use some context from the examples, but use the context and create your own 
            blog post. And please provide a lot of numbers and statistics.
            """
# print(user_prompt)
response = client.chat.completions.create(
                model="gpt-4o",
                messages=[
                    {"role": "system", "content": master_prompt},
                    {"role": "user", "content": user_prompt},
                ],
                max_tokens=4096,
                temperature=1.0,
            )
print(response)

ChatCompletion(id='chatcmpl-9qRAlxmHhu0hN0B4oXsgPjMmzQpht', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='{\n    "title": "How Rising Interest Rates and Market Shifts are Reshaping Home Buying in 2024",\n    "content": "## Introduction\n\nThe 2024 real estate market is navigating a substantial reshaping as rising interest rates and shifting market dynamics continue to influence home buying trends. With the Federal Reserve\'s ongoing adjustments and a rapidly evolving economic environment, prospective homeowners and investors must adapt to stay ahead. This article dives deep into current trends, home buying strategies, and the impact of interest rates on the real estate sector.\n\n## Current Real Estate Market Conditions\n\n### A Year in Review: 2023\n\nThe year 2023 saw pivotal changes in the real estate landscape marked by:\n\n- **Interest Rates:** The Federal Reserve increased interest rates multiple times, with the benchmark rat

In [3]:
print(response.choices[0].message.content)

{
    "title": "Navigating the 2024 Real Estate Landscape: Trends, Tips, and Insights",
    "content": "## The 2024 Real Estate Landscape: Trends, Tips, and Insights

The real estate market is constantly evolving, and as we step into 2024, there are a number of key trends and developments that both new buyers and seasoned investors need to be aware of. From shifts in market conditions to emerging technologies, staying informed can help you make better decisions and seize new opportunities. Here, we delve into the current state of the real estate market, offer valuable home buying and selling tips, and explore the latest in home improvement and interior design.

### Current Real Estate Market Conditions

The real estate market in 2024 is a mixed bag, reflecting both opportunities and challenges. Here are some highlights:

1. **Inventory Levels**: Inventory remains tight in many regions, particularly in desirable urban and suburban areas. This has kept prices relatively high, although th